# Machine Learning (MScA 32009)

# Introduction to TensorFlow: Part 1

## Yuri Balasanov, Mihail Tselishchev, &copy; iLykei 2018

##### Main text: Hands-On Machine Learning with Scikit-Learn and TensorFlow, Aurelien Geron, &copy; Aurelien Geron 2017, O'Reilly Media, Inc

# General Introduction

- TensorFlow was initially developed by Google Brain team for Google internal large-scale projects like Google Cloud Speech, Google Photos, Google Search, etc. <br>
- TensorFlow was designed particularly to train efficiently complex neural networks.
- In November 2015 Google made TensorFlow an open source product. By that time there were multiple libraries for neural networks available. Those libraries contained most of the features of TensorFlow. But the quality of the product, its clean design, scalability and flexibility together with well developed documentation quickly gained leadership among all similar products.

Attractive features of TensorFlow:

- Runs not only on Windows, Linux, macOS, but also on mobile devices with Android and iOS;
- Provides simple API TF.Learn (tensorflow.contrib.learn) compatible with Scikit-Learn. This previously had been an independent project Scikit Flow. TF.Learn allows training neural networks with minimum amount of code;
- Another API called TF-slim (tensorflow.contrib.slim) simplifies designing and evaluating neural networks;
- Several independently built on top of TensorFlow high-level APIs (among them Keras) made it one of the most popular open source projects on GitHub;
- TensorFlow is based on very efficient C++ library containing many popular Machine Learning methods;
- TensorFlow has Python as main API which combines high flexibility with computational efficiency;
- TensorFlow has very advanced gradient optimization methods based on automatic differentiation (autodiff);
- Google provided a very nice cloud service to run TensorFlow graphs


Main principle of TensorFlow: 

1. Define in Python a graph of computations (construction phase); 
2. Ask TensorFlow to run efficient calculation with this graph using optimized C++ software (execution phase). Calculations with the graph are parallelized between multiple CPUs, GPUs or TPUs.

In [1]:
import tensorflow as tf
t = tf.constant([[[],[]],[[],[]],[[],[]]])
print(t.shape)
print(t)

(3, 2, 0)
Tensor("Const:0", shape=(3, 2, 0), dtype=float32)


In [2]:
tf.reset_default_graph()

# Main components and visualization of graphs

### First node

Create a simple TensorFlow constant

In [3]:
s = tf.constant('Hello, World')
s

<tf.Tensor 'Const:0' shape=() dtype=string>

Note that 's' is not a string as it would be in pure python. <br>
Here 's' is a node in TensorFlow graph! <br>
Node is the main building block of the graph defining an operation.

The new node was added to the default graph.
To explore this graph, first find graph's [**operations**](https://www.tensorflow.org/api_docs/java/reference/org/tensorflow/Operation), i.e. nodes in graph that take zero or more tensors (produced by other operations in the graph) as input, and produce zero or more tensors as output.

Operation instances are valid only as long as the graph they are a part of is valid. Thus, if *close()* has been invoked, then methods on the operation instance may fail with an *IllegalStateException*.

Operation instances are immutable and thread-safe.  

In [4]:
graph = tf.get_default_graph()
print(graph)
graph.get_operations()

[<tf.Operation 'Const' type=Const>]

Method *'get_operations'* shows all operations of the graph. 
To show the representation of an operation use *'node_def'*.

In [5]:
graph.get_operations()[0].node_def

name: "Const"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_STRING
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_STRING
      tensor_shape {
      }
      string_val: "Hello, World"
    }
  }
}

In order to evaluate all the computations we need to create TensorFlow session object:

In [6]:
sess = tf.InteractiveSession()  # Create a session
msg = sess.run(s)  # Make computations
sess.close() # Don't forget to close the session!
print(msg)

b'Hello, World'


Reset the default graph before doing another series of calculations.

In [7]:
tf.reset_default_graph()

### Working with variables: construction phase

Workflow on any graph starts with definition of variables and constants.

Create variable $a=1.0$ and a constant $c=2.0$. <br>
Then change variable $a$ to 
$$a=a+c$$ 
and define new variable $b$ by simple formula
$$b=a*c$$

In [8]:
a = tf.Variable(1.0, name='a')  # create variable node in the graph with init value of 1.0
c = tf.constant(2.0)            # create constant value 2.0
a = a + c  # a := a+c
b = tf.multiply(a, c, 'output')  # b := a*c

Set name for the new default graph and find its operations.

In [9]:
graph = tf.get_default_graph()
graph.get_operations()

[<tf.Operation 'a/initial_value' type=Const>,
 <tf.Operation 'a' type=VariableV2>,
 <tf.Operation 'a/Assign' type=Assign>,
 <tf.Operation 'a/read' type=Identity>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'add' type=Add>,
 <tf.Operation 'output' type=Mul>]

This graph has more operations. <br>
We see that $a$ first took type "Const", then type "VariableV2", then it was assigned and read. <br>
Constant $c$ took type "Const". <br>
Then there is addition followed by multiplication.

Explore operation of reading variable $a$: input is $a$ and operation is "read".

In [10]:
# view variable init value node
graph.get_operations()[3].node_def

name: "a/read"
op: "Identity"
input: "a"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "_class"
  value {
    list {
      s: "loc:@a"
    }
  }
}

Operation of addition $a+c$ has 2 inputs: $a$ coming from "read" and constant $c$ 

In [11]:
graph.get_operations()[5].node_def

name: "add"
op: "Add"
input: "a/read"
input: "Const"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}

It is important to note that no calculations are executed until now. Not even assignment of values. <br>
**Only graph creation**. <br>
This is construction phase. <br>

It is possible to see the graph that was just created. <br>
For that add $a,b,c$ to the log.

In [12]:
tf.summary.scalar("a",a)
tf.summary.scalar("c",c)
tf.summary.scalar("b",b)

<tf.Tensor 'b:0' shape=() dtype=string>

Then write the log to a folder. Use unique folder name with time stamp.

In [13]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)  # path to TensorBoard logs
print(logdir)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

tf_logs/run-20180826_185541/


Now it is time to call TensorBoard, a very convenient tool for visualization. <br>
From terminal window or from the notebook execute

`tensorboard --logdir tf_logs/run-20171106_064428/`

Note that the path should be replaced with the path to your log file. <br>

After this opening page `http://localhost:6006` opens the graph visualization. <br>
Check each step on the graph and match it with the steps of the code.

It is recommended to close TensorBoard after working with visualization by `"CTRL+C"` in terminal window or by shutting down kernel of the Jupyter notebook. <br>

### Working with variables: execution phase

After graph is ready it can be executed. <br>
This requires opening TensorFlow session. <br>
TensorFlow session does placement of computations onto CPUs or GPUs and runs them.

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())   #initialize 'a', 'c' only now
    print(a.eval())                               #evaluate final 'a' and 'b' only now
    print(b.eval())

3.0
6.0


Working with session as shown above is convenient because inside the "with" block default TensorFlow session is set. This makes the code more readable. <br>
We also do not need to remember turning the session off at the end. <br>

However, it may be more convenient using an alternative handling of TensorFlow session if the code is in a Jupyter notebook. <br>

Using `sess = tf.InteractiveSession()` as below allows spreading the code through more than one cell.

In [15]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer()) # initialize all variables
print(sess.run([a,b]))

[3.0, 6.0]


Continue the session in the next cell and close it:

In [16]:
print(sess.run([a,b]))
sess.close()

[3.0, 6.0]


In [17]:
tf.reset_default_graph()

### Tensor variables

In [18]:
sess = tf.InteractiveSession()

Create a simple tensor (array).

In [19]:
t1 = tf.zeros([2,2])
print(t1[0])
print(t1[1])
t1.shape

Tensor("strided_slice:0", shape=(2,), dtype=float32)
Tensor("strided_slice_1:0", shape=(2,), dtype=float32)


TensorShape([Dimension(2), Dimension(2)])

Evaluate the tensor (synonim for sess.run(t1))

In [20]:
t1.eval(session=sess)

array([[ 0.,  0.],
       [ 0.,  0.]], dtype=float32)

Create tensor *'t2'* filled with values 3.14.

In [21]:
t2 = tf.fill(dims=[2,2], value=3.14, name='t1')
t2.eval(session=sess)

array([[ 3.1400001,  3.1400001],
       [ 3.1400001,  3.1400001]], dtype=float32)

Now add the variables *'t1'* and *'t2'*.

In [22]:
sess.run(t1+t2)

array([[ 3.1400001,  3.1400001],
       [ 3.1400001,  3.1400001]], dtype=float32)

Create a variable with initial value of matrix *'t2'*. <br>
Initiate this variable and square it. <br>

Function *'matmul()'* below is for matrix multiplication.

In [23]:
# Create a variable with init value of t2
t_var = tf.Variable(t2, name = 't_var')
sess.run(tf.variables_initializer([t_var])) # initialize variable
t_var = tf.matmul(t_var, t_var)
sess.run(t_var)

array([[ 19.71920204,  19.71920204],
       [ 19.71920204,  19.71920204]], dtype=float32)

In [24]:
sess.close()
tf.reset_default_graph()

### Placeholders logic

Placeholders are variables that are assigned values at a later time. <br>
Placeholders allow creating operations and build computation graphs, before the data are specified. <br>
In TensorFlow terminology, data then fed into the graph through these placeholders.

In [26]:
sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=(3, ))
y = tf.placeholder(tf.float32, shape=(3, ))
z = tf.add(x,y)
print(sess.run(z, feed_dict={x:[1,1,1],y:[2,2,2]}))  # here we assign explicit values for x and y nodes
sess.close()
tf.reset_default_graph()

[ 3.  3.  3.]


## Exercise

Create a function $f(x,y) = x^2y+y+2$, where values of $x,~y$ can be $x=3;~y=4$.

In [27]:
# Define variables x, y and f
x = tf.Variable(3,name="x")
y = tf.Variable(4, name="y")
f = x*x*y+y+2

Check the resulting graph.

In [28]:
# get_default_graph(), get_operations()
graph = tf.get_default_graph()
graph.get_operations()

[<tf.Operation 'x/initial_value' type=Const>,
 <tf.Operation 'x' type=VariableV2>,
 <tf.Operation 'x/Assign' type=Assign>,
 <tf.Operation 'x/read' type=Identity>,
 <tf.Operation 'y/initial_value' type=Const>,
 <tf.Operation 'y' type=VariableV2>,
 <tf.Operation 'y/Assign' type=Assign>,
 <tf.Operation 'y/read' type=Identity>,
 <tf.Operation 'mul' type=Mul>,
 <tf.Operation 'mul_1' type=Mul>,
 <tf.Operation 'add' type=Add>,
 <tf.Operation 'add_1/y' type=Const>,
 <tf.Operation 'add_1' type=Add>]

Create log and visualize the graph through tensorboard.

In [29]:
tf.summary.scalar("x",x)
tf.summary.scalar("y",y)
tf.summary.scalar("f",f)
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)  # path to TensorBoard logs
print(logdir)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

tf_logs/run-20180826_185629/


Explore the graph with *tensorboard*.

In [30]:
tf.reset_default_graph()